In [186]:
import pandas as pd
import numpy as np
import string
import datetime

In [187]:
# Load the Bronze layer data
df = pd.read_parquet('../Bronze/bronze_mario.parquet')


In [188]:
# Remove columns with too many missing values (more than 20% nulls)
def drop_null_columns(df, threshold=0.2):
    null_percentage = df.isnull().mean()
    columns_to_keep = null_percentage[null_percentage < threshold].index

    if len(columns_to_keep) == len(df.columns):
        print('No columns removed')
    else:
        print(f'removing: {[c for c in df.columns if c not in columns_to_keep]}')
    return df[columns_to_keep]

In [189]:
# Fill missing values in specified columns with default values
def fill_missing_values(df, columns_defaults: dict):
    for column, default_value in columns_defaults.items():
        df[column] = df[column].fillna(default_value)
    return df

In [190]:
# Convert columns to their correct data types
def convert_column_types(df, column_types: dict):
    try:
        for column, dtype in column_types.items():
            df[column] = df[column].astype(dtype)
        return df
    except Exception as e:
        print(f'{column} caused an issue')
        raise e


In [191]:
# Remove punctuation and underscores from specified text columns
def remove_punctuation(df, columns: list):
    for c in columns:
        df.loc[:,c] = df[c].str.replace(r'[^\w\s]|_','', regex=True)
    return df

df = remove_punctuation(df, ['Vehicle Type', 'World', 'Primary Game', 'Kart Role'])
df[['Vehicle Type', 'World', 'Primary Game', 'Kart Role']].head()


,Vehicle Type,World,Primary Game,Kart Role
0,Circuit Special,Toad Town,Super Mario World,Drifter
1,Comet Bike,Yoshis Island,Mario Kart 8 Deluxe,Blocker
2,Circuit Special,Yoshis Island,Mario Tennis Aces,Speedster
3,Scooter,Star World ...,Super Mario Odyssey,Item Specialist
4,Pipe Frame,Toad Town,Mario Strikers ...,Driver


In [192]:
# Fix minor spelling variation in Kart Role
df['Kart Role'] = df['Kart Role'].replace({'Drive': 'Driver'})

In [193]:
# Trip extra spaces
for col in ['Vehicle Type', 'Primary Game']:
    df[col] = (
        df[col].astype(str)
              .str.replace(r'\s+', ' ', regex=True)
              .str.strip()
    )

In [194]:
#Fix capitalization in key text columns
for col in ['Companion', 'Team', 'World']:
    df[col] = (
        df[col]
        .astype(str)              # ensure string type
        .str.strip()              # remove leading/trailing spaces
        .str.replace(r'\s+', ' ', regex=True)  # collapse double spaces
        .str.title()              # convert to Title Case (e.g., gOOMBA -> Goomba)
    )

# Quick preview
df[['Companion', 'Team', 'World']].head()

,Companion,Team,World
0,Toad,Green Caps,Toad Town
1,Koopa Troopa,Dino Buddies,Yoshis Island
2,Yoshi,Dino Buddies,Yoshis Island
3,Goomba,Sarasaland Stars,Star World
4,Goomba,Sarasaland Stars,Toad Town


In [195]:
# Check that each column matches its expected data type and report discrepancies
def check_formats(df, expected_formats: dict):
    incorrect_formats = []
    for column, datatype in df.dtypes.to_dict().items():
        expected_type = expected_formats.get(column)
        if expected_type != datatype:
            incorrect_formats.append((column, datatype, expected_type))

    incorrect_columns = [c[0] for c in incorrect_formats]
    correct_format_count = len([c for c in df.columns if c not in incorrect_columns])

    if incorrect_formats:
        print('Below are incorrect formats')
        print('_' * 50)
        print(f'Correct Column Count: {correct_format_count}')
        return pd.DataFrame(incorrect_formats, columns=['Column', 'Actual', 'Expected'])
    else:
        print('Validation Complete, no discrepancies')


In [196]:
# Calculate how similar two words are based on shared characters 
def check_similarity(word1: str, word2: str) -> float:
    word_set1 = set(word1)
    word_set2 = set(word2)
    intersection_count = len(word_set1.intersection(word_set2))
    total_char_count = len(word_set1.union(word_set2))
    similarity = intersection_count / total_char_count
    return similarity


In [197]:
# Find and list pairs of similar names in a column to detect possible misspellings

def check_misspelling(dataframe: pd.DataFrame, column: str, similarity_threshold: float) -> pd.DataFrame:
    all_unique_values = list(set(dataframe[column].tolist()))
    similarity_list = []

    for n in range(len(all_unique_values)):
        value1 = all_unique_values[n]
        for n2 in range(n + 1, len(all_unique_values)):
            value2 = all_unique_values[n2]
            similarity = round(check_similarity(value1, value2), 4)
            if similarity >= similarity_threshold:
                similarity_list.append([value1, value2, similarity])

    return pd.DataFrame(similarity_list, columns=['name1', 'name2', 'similarity'])


In [198]:
# Ensure Player Name has no nulls and is string before similarity check
df['Player Name'] = df['Player Name'].fillna('Unknown Name').astype(str)

In [199]:
# Produce & view the required table of similar player names (>= 0.80)
similar_names_tbl = check_misspelling(df, column='Player Name', similarity_threshold=0.80)
print(similar_names_tbl.head(20))

# Save for submission
similar_names_tbl.to_csv('./silver_similar_names_over_80.csv', index=False)

     name1   name2  similarity
0   Yoshiv  YosYhi      0.8333
1   Yoshiv  Yovshi      1.0000
2   Yoshiv  Yosshi      0.8333
3   Yoshiv  Yosihi      0.8333
4   Yoshiv  Yoshhi      0.8333
5   Yoshiv  YoshYi      0.8333
6   Yoshiv  Yoshis      0.8333
7   Yoshiv  YoshiY      0.8333
8   Yoshiv  Yhoshi      0.8333
9   Yoshiv  Yosvhi      1.0000
10  Yoshiv  Yoishi      0.8333
11  Yoshiv  Yvoshi      1.0000
12  Yoshiv  Yooshi      0.8333
13  Yoshiv  Yohshi      0.8333
14  Yoshiv  Yioshi      0.8333
15  Yoshiv   Yoshi      0.8333
16  LRuigi  LuigLi      0.8000
17  LRuigi  Luuigi      0.8000
18  LRuigi  Luiggi      0.8000
19  LRuigi  Luigui      0.8000


In [200]:
import difflib

valid_names = [
    "Mario","Luigi","Peach","Daisy","Yoshi","Toad","Toadette",
    "Rosalina","Wario","Waluigi","Bowser","Bowser Jr."
]

def correct_player_name(name):
    # Handle NaN/None/empty and non-strings
    if pd.isna(name):
        return "Unknown Name"
    name = str(name).strip()
    if name == "":
        return "Unknown Name"
    
    # Fuzzy match to the allowed list
    match = difflib.get_close_matches(name, valid_names, n=1, cutoff=0.8)
    return match[0] if match else name

# Ensure string dtype first
df['Player Name'] = df['Player Name'].astype('string')

df['Player Name'] = df['Player Name'].apply(correct_player_name)
print(df['Player Name'].unique())


['Bowser Jr.' 'Wario' 'Peach' 'Rosalina' 'Yoshi' 'Mario' 'Unknown Name'
 'Toadette' 'Daisy' 'Toad' 'Luigi' 'Waluigi' 'Bowser']


In [201]:
df.head()

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,...,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,fileName,loadDatetimeStamp
0,Bowser Jr.,Green Caps,Toad Town,Circuit Special,Toad,A,A,D,8,Drifter,...,0.0,No,Yes,"Green Shell, Fire Flower",83,51,1.000,Super Mario World,mario_data_20250601.jsonl,2025-10-24 15:56:48.529564
1,Bowser Jr.,Dino Buddies,Yoshis Island,Comet Bike,Koopa Troopa,A,S,A,28,Blocker,...,3.0,No,No,"Fire Flower, Super Mushroom",391,46,10.000,Mario Kart 8 Deluxe,mario_data_20250601.jsonl,2025-10-24 15:56:48.529564
2,Wario,Dino Buddies,Yoshis Island,Circuit Special,Yoshi,S,C,C,13,Speedster,...,0.0,Yes,None,"Fire Flower, Red Shell, Banana Peel",370,55,9.000,Mario Tennis Aces,mario_data_20250601.jsonl,2025-10-24 15:56:48.529564
3,Peach,Sarasaland Stars,Star World,Scooter,Goomba,B,D,B,7,Item Specialist,...,4.0,No,None,"Super Star, 1-Up Mushroom, Fire Flower",108,83,6.000,Super Mario Odyssey,mario_data_20250601.jsonl,2025-10-24 15:56:48.529564
4,Rosalina,Sarasaland Stars,Toad Town,Pipe Frame,Goomba,D,A,A,6,Driver,...,1.0,Yes,No,"Green Shell, Super Star",189,43,8.891,Mario Strikers,mario_data_20250601.jsonl,2025-10-24 15:56:48.529564


In [202]:
print(df.columns.tolist())

['Player Name', 'Team', 'World', 'Vehicle Type', 'Companion', 'Kart Racing Rank', 'Platforming Rank', 'Boss Battle Rank', 'Power-Ups Used', 'Kart Role', 'Team Points', 'Lives Lost', 'Participation in Battle Mode', 'Mushroom Cup Participation', 'Power-Ups Owned', 'Coins Spent in Toad Town', 'Levels Completed', 'Times Hit by Enemies', 'Primary Game', 'fileName', 'loadDatetimeStamp']


In [203]:
df.isnull().sum()

Player Name                         0
Team                                0
World                               0
Vehicle Type                        0
Companion                           0
Kart Racing Rank                 9280
Platforming Rank                    0
Boss Battle Rank                    0
Power-Ups Used                      0
Kart Role                           0
Team Points                         0
Lives Lost                          0
Participation in Battle Mode        0
Mushroom Cup Participation      12400
Power-Ups Owned                     0
Coins Spent in Toad Town            0
Levels Completed                    0
Times Hit by Enemies                0
Primary Game                        0
fileName                            0
loadDatetimeStamp                   0
dtype: int64

In [204]:
df.isnull().mean() * 100

Player Name                      0.000000
Team                             0.000000
World                            0.000000
Vehicle Type                     0.000000
Companion                        0.000000
Kart Racing Rank                19.333333
Platforming Rank                 0.000000
Boss Battle Rank                 0.000000
Power-Ups Used                   0.000000
Kart Role                        0.000000
Team Points                      0.000000
Lives Lost                       0.000000
Participation in Battle Mode     0.000000
Mushroom Cup Participation      25.833333
Power-Ups Owned                  0.000000
Coins Spent in Toad Town         0.000000
Levels Completed                 0.000000
Times Hit by Enemies             0.000000
Primary Game                     0.000000
fileName                         0.000000
loadDatetimeStamp                0.000000
dtype: float64

In [205]:
df = fill_missing_values(df, {
    'Player Name': 'Unknown Name',
    'Team': 'Unknown Team',
    'Kart Racing Rank': 'No Rank',
    'Mushroom Cup Participation': False
})


In [206]:
df.isnull().sum()

Player Name                     0
Team                            0
World                           0
Vehicle Type                    0
Companion                       0
Kart Racing Rank                0
Platforming Rank                0
Boss Battle Rank                0
Power-Ups Used                  0
Kart Role                       0
Team Points                     0
Lives Lost                      0
Participation in Battle Mode    0
Mushroom Cup Participation      0
Power-Ups Owned                 0
Coins Spent in Toad Town        0
Levels Completed                0
Times Hit by Enemies            0
Primary Game                    0
fileName                        0
loadDatetimeStamp               0
dtype: int64

In [207]:
# Define the expected schema for the dataset
expected_formats = {
    'Player Name': 'string',
    'Team': 'string',
    'World': 'string',
    'Vehicle Type': 'string',
    'Companion': 'string',
    'Kart Racing Rank': 'string',
    'Platforming Rank': 'string',
    'Boss Battle Rank': 'string',
    'Power-Ups Used': 'int32',
    'Kart Role': 'string',
    'Team Points': 'int32',
    'Lives Lost': 'int32',
    'Participation in Battle Mode': 'bool',
    'Mushroom Cup Participation': 'bool',
    'Power-Ups Owned': 'string',
    'Coins Spent in Toad Town': 'int32',
    'Levels Completed': 'int32',
    'Times Hit by Enemies': 'int32',
    'Primary Game': 'string',
    'fileName': 'string',
    'loadDatetimeStamp': 'datetime64[ns]',
}


In [208]:
# Convert loadDatetimeStamp column from text to datetime format for consistency
df['loadDatetimeStamp'] = pd.to_datetime(df['loadDatetimeStamp'], errors='coerce')

In [209]:
# Convert numeric columns to int32, replacing invalid or missing values with 0
numeric_cols = ['Power-Ups Used','Team Points','Lives Lost',
                'Coins Spent in Toad Town','Levels Completed','Times Hit by Enemies']
for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0).astype('int32')

In [210]:
# Convert 'Yes'/'No' text responses into True/False boolean values
yn_map = {'Yes': True, 'No': False, 'None': False, None: False, '': False}
df['Participation in Battle Mode'] = df['Participation in Battle Mode'].map(yn_map).fillna(False).astype('bool')
df['Mushroom Cup Participation']   = df['Mushroom Cup Participation'].map(yn_map).fillna(False).astype('bool')

/var/folders/bz/yy3kfr0n10d0z4rxjyx4y8dc0000gn/T/ipykernel_22789/417702394.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Mushroom Cup Participation']   = df['Mushroom Cup Participation'].map(yn_map).fillna(False).astype('bool')


In [211]:
# Convert remaining columns according to the schema
df = convert_column_types(df, expected_formats)

In [212]:
# Verify all columns have the correct data types; show mismatches if any
report = check_formats(df, expected_formats)
report if isinstance(report, pd.DataFrame) else print('Formats OK')

Validation Complete, no discrepancies
Formats OK


In [213]:
# Preview data, check for missing values, and verify correct data types before saving
df.info(), df.isnull().sum(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48000 entries, 0 to 47999
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Player Name                   48000 non-null  string        
 1   Team                          48000 non-null  string        
 2   World                         48000 non-null  string        
 3   Vehicle Type                  48000 non-null  string        
 4   Companion                     48000 non-null  string        
 5   Kart Racing Rank              48000 non-null  string        
 6   Platforming Rank              48000 non-null  string        
 7   Boss Battle Rank              48000 non-null  string        
 8   Power-Ups Used                48000 non-null  int32         
 9   Kart Role                     48000 non-null  string        
 10  Team Points                   48000 non-null  int32         
 11  Lives Lost                  

(None,
 Player Name                     0
 Team                            0
 World                           0
 Vehicle Type                    0
 Companion                       0
 Kart Racing Rank                0
 Platforming Rank                0
 Boss Battle Rank                0
 Power-Ups Used                  0
 Kart Role                       0
 Team Points                     0
 Lives Lost                      0
 Participation in Battle Mode    0
 Mushroom Cup Participation      0
 Power-Ups Owned                 0
 Coins Spent in Toad Town        0
 Levels Completed                0
 Times Hit by Enemies            0
 Primary Game                    0
 fileName                        0
 loadDatetimeStamp               0
 dtype: int64,
   Player Name              Team          World     Vehicle Type     Companion  \
 0  Bowser Jr.        Green Caps      Toad Town  Circuit Special          Toad   
 1  Bowser Jr.      Dino Buddies  Yoshis Island       Comet Bike  Koopa Troopa 

In [214]:
# Keep only valid Mario characters for Player Name
valid_names = [
    "Mario","Luigi","Peach","Daisy","Yoshi","Toad","Toadette",
    "Rosalina","Wario","Waluigi","Bowser","Bowser Jr."
]

df = df[df['Player Name'].isin(valid_names)].copy()

# Confirm the filter worked
print(sorted(df['Player Name'].unique()))
print(f"✅ Remaining rows after filtering: {len(df)}")

['Bowser', 'Bowser Jr.', 'Daisy', 'Luigi', 'Mario', 'Peach', 'Rosalina', 'Toad', 'Toadette', 'Waluigi', 'Wario', 'Yoshi']
✅ Remaining rows after filtering: 38960


In [215]:
# Save the cleaned and validated Silver dataset to parquet format
df.to_parquet('./silver_mario.parquet', index=False)